In [249]:
import pandas as pd
import spacy
from spacy.matcher import PhraseMatcher

nlp = spacy.load('en_core_web_sm')
docs = nlp.tokenizer("how is it going i'll")

x = [token for token in docs]
x

how is it going i'll

In [231]:
# Load in the data from JSON file
data = pd.read_json('restaurant.json')
data.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
109,lDJIaF4eYRF4F7g6Zb9euw,lb0QUR5bc4O-Am4hNq9ZGg,r5PLDU-4mSbde5XekTXSCA,4,2,0,0,I used to work food service and my manager at ...,2013-01-27 17:54:54
1013,vvIzf3pr8lTqE_AOsxmgaA,MAmijW4ooUzujkufYYLMeQ,r5PLDU-4mSbde5XekTXSCA,4,0,0,0,We have been trying Eggplant sandwiches all ov...,2015-04-15 04:50:56
1204,UF-JqzMczZ8vvp_4tPK3bQ,slfi6gf_qEYTXy90Sw93sg,r5PLDU-4mSbde5XekTXSCA,5,1,0,0,Amazing Steak and Cheese... Better than any Ph...,2011-03-20 00:57:45
1251,geUJGrKhXynxDC2uvERsLw,N_-UepOzAsuDQwOUtfRFGw,r5PLDU-4mSbde5XekTXSCA,1,0,0,0,Although I have been going to DeFalco's for ye...,2018-07-17 01:48:23
1354,aPctXPeZW3kDq36TRm-CqA,139hD7gkZVzSvSzDPwhNNw,r5PLDU-4mSbde5XekTXSCA,2,0,0,0,"Highs: Ambience, value, pizza and deserts. Thi...",2018-01-21 10:52:58


In [232]:
menu = ["Cheese Steak", "Cheesesteak", "Steak and Cheese", "Italian Combo", "Tiramisu", "Cannoli",
        "Chicken Salad", "Chicken Spinach Salad", "Meatball", "Pizza", "Pizzas", "Spaghetti",
        "Bruchetta", "Eggplant", "Italian Beef", "Purista", "Pasta", "Calzones",  "Calzone",
        "Italian Sausage", "Chicken Cutlet", "Chicken Parm", "Chicken Parmesan", "Gnocchi",
        "Chicken Pesto", "Turkey Sandwich", "Turkey Breast", "Ziti", "Portobello", "Reuben",
        "Mozzarella Caprese",  "Corned Beef", "Garlic Bread", "Pastrami", "Roast Beef",
        "Tuna Salad", "Lasagna", "Artichoke Salad", "Fettuccini Alfredo", "Chicken Parmigiana",
        "Grilled Veggie", "Grilled Veggies", "Grilled Vegetable", "Mac and Cheese", "Macaroni",  
         "Prosciutto", "Salami"]

In [233]:
index_of_review_to_test_on = 1
text_to_test_on = data.text.iloc[index_of_review_to_test_on]

text_to_test_on

"We have been trying Eggplant sandwiches all over the valley because my day loves eggplant.  Delfaco's sandwich on the Italian Hoagie had way way too much bread more bread than eggplant.  The eggplant by itself was very tasty, but get it without so much bread, you will thank me later.  We will be back to try something else.  I went at 11 am on a Saturday morning and still waited over 30 minutes for a take out order.  It is a very crammed space, a little Italian deli in between a Italian grocery store.  The prices for the Italian pastas and cheese are a bit over priced.  The espresso is nicely priced at $1.40 for a single shot."

In [234]:
# Create the tokenized version of text_to_test_on
review_doc = nlp(text_to_test_on)
for token in review_doc:
    print(f"{token.text} \t\t\t {token.lemma_} \t\t\t {token.is_stop}")

We 			 -PRON- 			 True
have 			 have 			 True
been 			 be 			 True
trying 			 try 			 False
Eggplant 			 eggplant 			 False
sandwiches 			 sandwich 			 False
all 			 all 			 True
over 			 over 			 True
the 			 the 			 True
valley 			 valley 			 False
because 			 because 			 True
my 			 -PRON- 			 True
day 			 day 			 False
loves 			 love 			 False
eggplant 			 eggplant 			 False
. 			 . 			 False
  			   			 False
Delfaco 			 Delfaco 			 False
's 			 's 			 True
sandwich 			 sandwich 			 False
on 			 on 			 True
the 			 the 			 True
Italian 			 Italian 			 False
Hoagie 			 Hoagie 			 False
had 			 have 			 True
way 			 way 			 False
way 			 way 			 False
too 			 too 			 True
much 			 much 			 True
bread 			 bread 			 False
more 			 more 			 True
bread 			 bread 			 False
than 			 than 			 True
eggplant 			 eggplant 			 False
. 			 . 			 False
  			   			 False
The 			 the 			 True
eggplant 			 eggplant 			 False
by 			 by 			 True
itself 			 -PRON- 			 True
was 			 be 			 True
very 			

In [235]:
# Create the PhraseMatcher object. The tokenizer is the first argument. Use attr = 'LOWER' to make consistent capitalization
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')

# Create a list of tokens for each item in the menu. The list contains a collection of "doc" objects
menu_tokens_list = [nlp(item) for item in menu]


In [236]:
matcher.add("MenuItems", menu_tokens_list)

items_in_review = matcher(review_doc) #Get the list of menu items that are in the review
print(items_in_review)

[(533989841420491665, 4, 5), (533989841420491665, 14, 15), (533989841420491665, 33, 34), (533989841420491665, 37, 38)]


In [237]:
for match_id, start, end in items_in_review:
    print("token->", start, review_doc[start:end])
    
for match in items_in_review:
    print(match[0], "/", match[1], "/", match[2])

token-> 4 Eggplant
token-> 14 eggplant
token-> 33 eggplant
token-> 37 eggplant
533989841420491665 / 4 / 5
533989841420491665 / 14 / 15
533989841420491665 / 33 / 34
533989841420491665 / 37 / 38


In [ ]:
#Now we will create a dictionary of all items and their ratings which can be used for modeling
from collections import defaultdict

item_ratings = defaultdict(list) #dictionary of item: rating list e.g. {'eggplant' : [1,2,4,5,5,4,3,3]}

for idx, review in data.iterrows():
    #print(idx, review.stars)
    doc = nlp(review.text) #convert user review text into an NLP doc
    items_in_review = matcher(doc) #find all menu items referenced in user review
    
    #doc[match1:match2] gets the name of menu item
    found_items = set([doc[match[1]:match[2]].lower_ for match in items_in_review])
    #print(found_items)

    for item in found_items:
        item_ratings[item].append(review.stars)


In [242]:
mean_ratings = {} #dictionary to store mean values - menu_item : mean_rating

#for item in item_ratings:
#    mean_ratings[item] = sum(item_ratings[item]) / len(item_ratings[item])

#list comprehension for-loop is used to simplify the code, but works exactly as 2 lines above.
mean_ratings = {item:sum(item_ratings[item]) / len(item_ratings[item]) for item in item_ratings}
    
print(mean_ratings)


{'chicken parmigiana': 4.470588235294118, 'eggplant': 4.159420289855072, 'pizza': 4.339622641509434, 'steak and cheese': 4.888888888888889, 'meatball': 4.1796875, 'cannoli': 4.388888888888889, 'pasta': 4.407766990291262, 'purista': 4.666666666666667, 'prosciutto': 4.68, 'cheese steak': 4.447368421052632, 'cheesesteak': 4.484536082474227, 'calzone': 4.444444444444445, 'italian combo': 4.0476190476190474, 'tiramisu': 4.238095238095238, 'chicken spinach salad': 4.5, 'italian beef': 3.92, 'salami': 4.25, 'chicken parm': 4.22, 'ziti': 4.380952380952381, 'chicken cutlet': 3.4, 'turkey sandwich': 3.8, 'tuna salad': 4.0, 'chicken pesto': 4.555555555555555, 'artichoke salad': 5.0, 'lasagna': 4.4576271186440675, 'fettuccini alfredo': 5.0, 'pizzas': 4.375, 'turkey breast': 5.0, 'calzones': 4.542857142857143, 'grilled veggie': 4.5, 'mac and cheese': 4.454545454545454, 'garlic bread': 4.128205128205129, 'spaghetti': 3.888888888888889, 'italian sausage': 4.30188679245283, 'portobello': 4.5, 'pastram

In [243]:
worst_item = sorted(mean_ratings, key=mean_ratings.get)[0]

print(worst_item)
print(mean_ratings[worst_item])

chicken cutlet
3.4


In [244]:
#Count of ratings per menu item
freq = {item: len(ratings) for item, ratings in item_ratings.items()}
#print(freq)
most_reviews = sorted(freq, key=freq.get, reverse=True)

for item in most_reviews:
    print(f"ggg {item} / {freq[item]}")


ggg pizza / 265
ggg pasta / 206
ggg meatball / 128
ggg cheesesteak / 97
ggg cheese steak / 76
ggg cannoli / 72
ggg calzone / 72
ggg eggplant / 69
ggg purista / 63
ggg lasagna / 59
ggg italian sausage / 53
ggg prosciutto / 50
ggg chicken parm / 50
ggg garlic bread / 39
ggg gnocchi / 37
ggg spaghetti / 36
ggg calzones / 35
ggg pizzas / 32
ggg salami / 28
ggg chicken pesto / 27
ggg italian beef / 25
ggg italian combo / 21
ggg tiramisu / 21
ggg ziti / 21
ggg chicken parmesan / 19
ggg chicken parmigiana / 17
ggg portobello / 14
ggg mac and cheese / 11
ggg chicken cutlet / 10
ggg steak and cheese / 9
ggg pastrami / 9
ggg roast beef / 7
ggg fettuccini alfredo / 6
ggg grilled veggie / 6
ggg turkey sandwich / 5
ggg tuna salad / 5
ggg artichoke salad / 5
ggg macaroni / 5
ggg chicken salad / 5
ggg reuben / 4
ggg chicken spinach salad / 2
ggg corned beef / 2
ggg turkey breast / 1
